In [ ]:
import numpy as np
import pandas as pd
import os

import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPool2D, BatchNormalization

import matplotlib.pyplot as plt

In [ ]:
dir = '/leapGestRecog'

In [ ]:
images = []
labels = []
for directory in os.listdir(dir):
  for subDir in os.listdir(os.path.join(dir,directory)):
    for img in os.listdir(os.path.join(dir, directory, subDir)):
      img_path = os.path.join(dir, directory, subDir, img)
      images.append(img_path)
      labels.append(subDir)

In [ ]:
Iseries = pd.Series(images, name="Images")
Lseries = pd.Series(labels, name="labels")
hand_gesture_data = pd.concat([Iseries, Lseries], axis=1)
hand_gesture_df = pd.DataFrame(hand_gesture_data)

In [ ]:
X_train, X_test = train_test_split(hand_gesture_df, test_size=0.2, random_state=42)
train_set, val_set = train_test_split(hand_gesture_df, test_size=0.3, random_state=42)

In [ ]:
image_gen = ImageDataGenerator(preprocessing_function= tf.keras.applications.mobilenet_v2.preprocess_input)
train = image_gen.flow_from_dataframe(dataframe= train_set,x_col="Images",y_col="labels",
                                      target_size=(244,244),
                                      color_mode='rgb',
                                      class_mode="categorical",
                                      batch_size=4,
                                      shuffle=False
                                     )
test = image_gen.flow_from_dataframe(dataframe= X_test,x_col="Images", y_col="labels",
                                     target_size=(244,244),
                                     color_mode='rgb',
                                     class_mode="categorical",
                                     batch_size=4,
                                     shuffle= False
                                    )
val = image_gen.flow_from_dataframe(dataframe= val_set,x_col="Images", y_col="labels",
                                    target_size=(244,244),
                                    color_mode= 'rgb',
                                    class_mode="categorical",
                                    batch_size=4,
                                    shuffle=False
                                   )

Found 14000 validated image filenames belonging to 10 classes.

Found 4000 validated image filenames belonging to 10 classes.

Found 6000 validated image filenames belonging to 10 classes.

In [ ]:
classes=list(train.class_indices.keys())
print (classes)

['01_palm', '02_l', '03_fist', '04_fist_moved', '05_thumb', '06_index', '07_ok', '08_palm_moved', '09_c', '10_down']

In [ ]:
def show_hand_gesture(image_gen):
    test_dict = test.class_indices
    classes = list(test_dict.keys())
    images, labels=next(image_gen)
    plt.figure(figsize=(20,20))
    length = len(labels)
    if length<25:
        r=length
    else:
        r=25
    for i in range(r):
        plt.subplot(5,5,i+1)
        image=(images[i]+1)/2
        plt.imshow(image)
        index=np.argmax(labels[i])
        class_name=classes[index]
        plt.title(class_name, color="green",fontsize=16)
        plt.axis('off')
    plt.show()
show_hand_gesture(train)

In [ ]:
 model = Sequential([
    Conv2D(filters=128, kernel_size=(8, 8), strides=(3, 3), activation='relu', input_shape=(224, 224, 3)),
    BatchNormalization(),

    Conv2D(filters=256, kernel_size=(5, 5), strides=(1, 1), activation='relu', padding="same"),
    BatchNormalization(),
    MaxPool2D(pool_size=(3, 3)),

    Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1), activation='relu', padding="same"),
    BatchNormalization(),
    Conv2D(filters=256, kernel_size=(1, 1), strides=(1, 1), activation='relu', padding="same"),
    BatchNormalization(),
    Conv2D(filters=256, kernel_size=(1, 1), strides=(1, 1), activation='relu', padding="same"),
    BatchNormalization(),

    Conv2D(filters=512, kernel_size=(3, 3), activation='relu', padding="same"),
    BatchNormalization(),
    MaxPool2D(pool_size=(2, 2)),

    Conv2D(filters=512, kernel_size=(3, 3), activation='relu', padding="same"),
    BatchNormalization(),

    Conv2D(filters=512, kernel_size=(3, 3), activation='relu', padding="same"),
    BatchNormalization(),

    MaxPool2D(pool_size=(2, 2)),

    Conv2D(filters=512, kernel_size=(3, 3), activation='relu', padding="same"),
    BatchNormalization(),

    MaxPool2D(pool_size=(2, 2)),

    Flatten(),
    Dense(1024, activation='relu'),
    Dropout(0.5),
    Dense(1024, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
])

model.compile(
    loss='categorical_crossentropy',
    optimizer=tf.optimizers.SGD(learning_rate=0.001),
    metrics=['accuracy']
)

model.summary()

Model: "sequential_2"

_________________________________________________________________

 Layer (type)                Output Shape              Param #   

=================================================================

 conv2d_18 (Conv2D)          (None, 73, 73, 128)       24704     

                                                                 

 batch_normalization_18 (Ba  (None, 73, 73, 128)       512       

 tchNormalization)                                               

                                                                 

 conv2d_19 (Conv2D)          (None, 73, 73, 256)       819456    

                                                                 

 batch_normalization_19 (Ba  (None, 73, 73, 256)       1024      

 tchNormalization)                                               

                                                                 

 max_pooling2d_8 (MaxPoolin  (None, 24, 24, 256)       0         

 g2D)                                                            

                                                                 

 conv2d_20 (Conv2D)          (None, 24, 24, 256)       590080    

                                                                 

 batch_normalization_20 (Ba  (None, 24, 24, 256)       1024      

 tchNormalization)                                               

                                                                 

 conv2d_21 (Conv2D)          (None, 24, 24, 256)       65792     

                                                                 

 batch_normalization_21 (Ba  (None, 24, 24, 256)       1024      

 tchNormalization)                                               

                                                                 

 conv2d_22 (Conv2D)          (None, 24, 24, 256)       65792     

                                                                 

 batch_normalization_22 (Ba  (None, 24, 24, 256)       1024      

 tchNormalization)                                               

                                                                 

 conv2d_23 (Conv2D)          (None, 24, 24, 512)       1180160   

                                                                 

 batch_normalization_23 (Ba  (None, 24, 24, 512)       2048      

 tchNormalization)                                               

                                                                 

 max_pooling2d_9 (MaxPoolin  (None, 12, 12, 512)       0         

 g2D)                                                            


In [ ]:
pred = model.predict(test)
pred = np.argmax(pred, axis=1)

labels = (train.class_indices)
labels = dict((v,k) for k,v in labels.items())
pred2 = [labels[k] for k in pred]



In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

y_test = X_test.labels
print(classification_report(y_test, pred2))
print("Accuracy of the Model:","{:.1f}%".format(accuracy_score(y_test, pred2)*100))

        precision    recall  f1-score   support



      01_palm       1.00      1.00      1.00       404

         02_l       1.00      1.00      1.00       377

      03_fist       1.00      1.00      1.00       404

04_fist_moved       1.00      1.00      1.00       410

     05_thumb       1.00      1.00      1.00       417

     06_index       1.00      1.00      1.00       366

        07_ok       1.00      1.00      1.00       418

08_palm_moved       1.00      1.00      1.00       403

         09_c       1.00      1.00      1.00       392

      10_down       1.00      1.00      1.00       409



     accuracy                           1.00      4000

    macro avg       1.00      1.00      1.00      4000

 weighted avg       1.00      1.00      1.00      4000



Accuracy of the Model: 100.0%